## Task One: Binary Representations

##### This task was all about working with bits. I had to implement four functions that manipulate bits in a 32-bit unsigned integer. These kinds of operations are essential in areas like cryptography and data compression, where you need to optimize data handling at the binary level.

### **Functions Implemented**
- **`rotl(x, n)`** – Rotates bits to the left.
- **`rotr(x, n)`** – Rotates bits to the right.
- **`ch(x, y, z)`** – Conditional bitwise selection.
- **`maj(x, y, z)`** – Computes the majority bit at each position.
##### Each function is essential in bitwise manipulations, particularly in hashing algorithms (e.g., SHA-256), data encoding, and optimizing storage.

### 1. **`rotl(x, n)`**
The rotate left (rotl) function shifts the bits of a number to the left by n positions. Bits that overflow on the left wrap around to the right. This is different from a left shift (<<), where overflowed bits are lost.

##### Why This Approach?
- Bitwise Shifting & Masking
Python supports [bitwise operations](https://docs.python.org/3/library/stdtypes.html#bitwise-operations) for manipulating integers efficiently. Since Python integers are arbitrary precision, we use bitwise masking (& 0xFFFFFFFF) to limit the result to 32 bits, ensuring behavior similar to C-based implementations.

- Handling Overflow Correctly
Since a [left shift](https://stackoverflow.com/questions/141525/what-are-bitwise-shift-bit-shift-operators-and-how-do-they-work) (`<<`) moves bits out of the range, I needed to reintroduce the overflowed bits using a right shift (`>>`) and a [bitwise OR](https://stackoverflow.com/questions/17484720/how-do-bitwise-or-and-bitwise-and-work-in-python) (`|`) to complete the rotation.




In [99]:

def rotl(x: int, n: int = 1) -> int:
    """
    Rotates the bits of a 32-bit unsigned integer to the left by n positions.

    Parameters:
        x (int): The 32-bit unsigned integer to rotate.
        n (int): The number of positions to rotate (default is 1).

    Returns:
        int: The rotated 32-bit unsigned integer.
    """
    n = n % 32  # Ensure n is within the valid range (0-31)
    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF  # Perform bitwise rotation


- Example Usage Of Rotl
- This example rotates the bits of a number to the left by 4 positions and shows the result, including a case where all bits are set.


In [100]:
# Example usage of rotl
x = 0b00000000000000000000000000000001  # Binary representation of 1
rotated_value = rotl(x, 4)

# Output the original and rotated values
print(f"Original value (bin): {bin(x)}")
print(f"Rotated left by 4 (bin): {bin(rotated_value)}")

# Example with all bits set
x_all_set = 0xFFFFFFFF  # All 32 bits set to 1
rotated_all_set = rotl(x_all_set, 5)
print(f"Rotated left (all bits set): {bin(rotated_all_set)}") 


Original value (bin): 0b1
Rotated left by 4 (bin): 0b10000
Rotated left (all bits set): 0b11111111111111111111111111111111


#### Test Case For Rotate Left Function
##### The tests ensure the rotl function correctly rotates bits in a 32-bit unsigned integer:

- Basic Rotation: Checks if a simple left rotation shifts bits correctly.
- Wraparound Behavior: Verifies that overflowed bits wrap around to the right.
- Zero Rotation: Ensures rotating by 0 positions returns the original value.


In [ ]:
import unittest 

class TestRotateLeft(unittest.TestCase):
    """
    Unit test class for testing the rotl (rotate left) function.
    """

    def test_rotl_basic(self):
        """
        Test case 1: Basic left rotation.
        - Input: `0b0001` (binary 1)
        - Rotating left by 4 positions
        - Expected Output: `0b10000` (binary 16)
        """
        self.assertEqual(rotl(0b0001, 4), 0b10000)

    def test_rotl_wraparound(self):
        """
        Test case 2: Check bit wraparound behavior.
        - Input: `0b10000000000000000000000000000000` (binary, MSB set)
        - Rotating left by 1 position
        - Expected Output: `0b00000000000000000000000000000001` (binary, LSB set)
        """
        self.assertEqual(rotl(0b10000000000000000000000000000000, 1), 
                         0b00000000000000000000000000000001)

    def test_rotl_zero_rotation(self):
        """
        Test case 3: Zero rotation (no changes).
        - Input: `0b1010` (binary 10)
        - Rotating left by 0 positions
        - Expected Output: `0b1010` (unchanged)
        """
        self.assertEqual(rotl(0b1010, 0), 0b1010)

# Run the tests
unittest.main(argv=[''], verbosity=2, exit=False)


### 2. **`rotr(x, n)`** 
The rotate right function shifts bits to the right by n positions. Bits that overflow on the right wrap around to the left.This ensures the integrity of data, unlike a right shift (>>), which discards shifted bits.

##### Why This Approach?
- Bitwise Shifting & Masking
Python supports [bitwise operations](https://docs.python.org/3/library/stdtypes.html#bitwise-operations) for manipulating integers efficiently. Since Python integers are arbitrary precision, we use bitwise masking (& 0xFFFFFFFF) to limit the result to 32 bits, ensuring behavior similar to C-based implementations.

- Handling Overflow Correctly
Since a [right shift](https://docs.python.org/3/library/stdtypes.html#bitwise-operations) (`>>`) moves bits out of range, I needed to reintroduce the overflowed bits using a [left shift](https://en.wikipedia.org/wiki/Circular_shift) (`<<`) and a [bitwise](https://en.wikipedia.org/wiki/Bitwise_operation#OR) OR (`|`) to complete the rotation and maintain proper circular shifting.


In [102]:
def rotr(x: int, n: int = 1) -> int:
    """
    Rotates the bits of a 32-bit unsigned integer to the right by n positions.

    Parameters:
        x (int): The 32-bit unsigned integer to rotate.
        n (int): The number of positions to rotate (default is 1).

    Returns:
        int: The rotated 32-bit unsigned integer.
    """
    n = n % 32  # Ensure n is within the valid range (0-31)
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF  # Perform bitwise rotation


##### Example Usage Of Rotr 
This example demonstrates rotating a number's bits to the right by 1 and 2 positions, including a case with a repeating bit pattern.


In [103]:
# Example usage of rotr
x = 0b10000000000000000000000000000000  # Binary representation with only the most significant bit set
rotated_value = rotr(x, 1)

# Output the original and rotated values
print(f"Original value (bin): {bin(x)}")
print(f"Rotated right by 1 (bin): {bin(rotated_value)}")

# Example with a pattern
x_pattern = 0b11001100110011001100110011001100
rotated_pattern = rotr(x_pattern, 2)
print(f"Original value (bin): {bin(x_pattern)}") 
print(f"Rotated right by 2 (bin): {bin(rotated_pattern)}")


Original value (bin): 0b10000000000000000000000000000000
Rotated right by 1 (bin): 0b1000000000000000000000000000000
Original value (bin): 0b11001100110011001100110011001100
Rotated right by 2 (bin): 0b110011001100110011001100110011


#### Test Case For Rotate Right Function
##### The tests ensure the rotr function correctly rotates bits in a 32-bit unsigned integer:

- Basic Rotation: Checks if a simple right rotation shifts bits correctly.
- Wraparound Behavior: Verifies that overflowed bits wrap around to the left.
- Zero Rotation: Ensures rotating by 0 positions returns the original value

In [ ]:
import unittest

class TestRotateRight(unittest.TestCase):

    def test_rotr_basic(self):
        """Test basic right rotation by 1 position."""
        self.assertEqual(rotr(0b1000, 1), 0b0100)

    def test_rotr_wraparound(self):
        """Test wrap-around where the least significant bit moves to the most significant bit."""
        self.assertEqual(rotr(0b00000000000000000000000000000001, 1), 
                         0b10000000000000000000000000000000)

    def test_rotr_zero_rotation(self):
        """Test rotation by 0 positions (should return the original value)."""
        self.assertEqual(rotr(0b1010, 0), 0b1010)

unittest.main(argv=[''], verbosity=2, exit=False)



### 3. **`ch(x, y, z)`** 
##### The choose function (ch) is commonly used in cryptographic hashing (SHA-256). It selects bits from y or z based on x:

##### - If x has a 1, take the bit from y.
##### - If x has a 0, take the bit from z.


In [105]:
def ch(x: int, y: int, z: int) -> int:
    """
    Chooses bits from y where x has 1s, and from z where x has 0s.

    Parameters:
        x (int): Selector bits.
        y (int): Bits selected when x is 1.
        z (int): Bits selected when x is 0.

    Returns:
        int: The resulting integer after selection.
    """
    return (x & y) | (~x & z) & 0xFFFFFFFF  # Perform bitwise selection



### Example Usage Of CH
##### This example demonstrates the ch function, which selects bits from y where x has 1s and from z where x has 0s, producing a result based on the selector bits in x.


In [106]:
# Example usage of ch
x = 0b10101010  # Selector bits
y = 0b11111111  # Bits to choose when x has 1s
z = 0b00000000  # Bits to choose when x has 0s

chosen_bits = ch(x, y, z)

# Output the result of the choose function
print(f"Selector (x):   {bin(x)}") 
print(f"Option 1 (y):   {bin(y)}")
print(f"Option 2 (z):   {bin(z)}")
print(f"Chosen result:  {bin(chosen_bits)}")

Selector (x):   0b10101010
Option 1 (y):   0b11111111
Option 2 (z):   0b0
Chosen result:  0b10101010


### - Test Case for CH Function


In [ ]:
import unittest

class TestChooseFunction(unittest.TestCase):

    def test_ch_basic(self):
        """Test basic choose function with a mix of 1s and 0s in the selector."""
        self.assertEqual(ch(0b10101010, 0b11111111, 0b00000000), 0b10101010)

    def test_ch_all_ones_selector(self):
        """Test when the selector is all 1s (should return y)."""
        self.assertEqual(ch(0b11111111, 0b10101010, 0b01010101), 0b10101010)

    def test_ch_all_zeros_selector(self):
        """Test when the selector is all 0s (should return z)."""
        self.assertEqual(ch(0b00000000, 0b10101010, 0b01010101), 0b01010101)

    def test_ch_alternating_pattern(self):
        """Test with alternating selector bits."""
        self.assertEqual(ch(0b11001100, 0b11110000, 0b00001111), 0b11000011)

unittest.main(argv=[''], verbosity=2, exit=False)



### 4. **`maj(x, y, z)`** 
##### The majority function (maj) is widely used in cryptography and error correction:

##### - If at least two of x, y, z have a 1 at a bit position, the result is 1.
##### - Otherwise, the result is 0.


In [108]:
def maj(x: int, y: int, z: int) -> int:
    """
    Computes the majority function at each bit position.

    Parameters:
        x (int): First input.
        y (int): Second input.
        z (int): Third input.

    Returns:
        int: The majority result.
    """
    return (x & y) | (x & z) | (y & z) & 0xFFFFFFFF  # Compute majority bitwise


### Example Usage Of MAJ
##### This example demonstrates the maj function, which outputs a 1 in each bit position where at least two of the inputs have a 1, showing how the majority vote works at the bit level.

In [109]:
# Example usage of maj
x = 0b10101010  # Input 1
y = 0b11110000  # Input 2
z = 0b00001111  # Input 3

majority_bits = maj(x, y, z)

# Output the result of the majority function
print(f"Input 1 (x):    {bin(x)}") 
print(f"Input 2 (y):    {bin(y)}")
print(f"Input 3 (z):    {bin(z)}")
print(f"Majority result: {bin(majority_bits)}")


Input 1 (x):    0b10101010
Input 2 (y):    0b11110000
Input 3 (z):    0b1111
Majority result: 0b10101010


### - Test Case For MAJ

In [ ]:
import unittest

class TestMajorityFunction(unittest.TestCase):

    def test_maj_basic(self):
        """Test majority function with mixed input bits."""
        self.assertEqual(maj(0b10101010, 0b11110000, 0b00001111), 0b10101010)

    def test_maj_all_zeros(self):
        """Test when all inputs are zeros (result should be all zeros)."""
        self.assertEqual(maj(0b00000000, 0b00000000, 0b00000000), 0b00000000)

    def test_maj_all_ones(self):
        """Test when all inputs are ones (result should be all ones)."""
        self.assertEqual(maj(0b11111111, 0b11111111, 0b11111111), 0b11111111)

    def test_maj_two_majority_bits(self):
        """Test when two inputs agree on bits and the third differs."""
        self.assertEqual(maj(0b10101010, 0b10101010, 0b01010101), 0b10101010)

    def test_maj_pattern_mixed_bits(self):
        """Test a case with a mix of majority and minority bits."""
        self.assertEqual(maj(0b11001100, 0b11110000, 0b00001111), 0b11001100)

unittest.main(argv=[''], verbosity=2, exit=False)


## Task Two: Hash Functions

### Overview
##### This task explores hash functions, which are widely used in cryptography, data integrity, and efficient data lookup.

- Convert a given C hash function to Python.
- Test the Python implementation with different inputs.
- Explain why the numbers 31 and 101 were chosen in the function.

##### A hash function takes an input eg.a string and converts it into a fixed-size integer value. It should be:

- Efficient → Quickly compute a unique hash for an input.
- Deterministic → The same input always produces the same hash.
- Uniform → Hash values should be well distributed to minimize collisions.

### Understanding the C Function and Translation Process
##### The provided C function computes a hash value for a given string using a weighted sum approach and a modulo operation. The goal is to convert this function into Python while maintaining its logic and efficiency




```
The original C function:

unsigned hash(char *s) {
    unsigned hashval;
    for (hashval = 0; *s != '\0'; s++)
        hashval = *s + 31 * hashval;
    return hashval % 101;
}
```

### Translating to Python
Python does not use pointers, so i needed to adapt this function accordingly:

Key Adaptations:
- Use ord(char) to get the ASCII value (since char *s in C directly accesses ASCII).
- Iterate through the string using for char in s: instead of pointer arithmetic (*s != '\0').
- Keep integer calculations the same (31 * hashval + ASCII).
- Apply modulo 101 at the end to match the behavior.

In [113]:
def hash_function(s: str) -> int:
    """
    Implements the given C hash function in Python.

    Parameters:
        s (str): Input string.

    Returns:
        int: Hash value mod 101.
    """
    hashval = 0  # Initialize hash value
    for char in s:
        hashval = ord(char) + 31 * hashval  # Apply weighted sum with ASCII value
    return hashval % 101  # Use modulo to limit range

# Example test cases
test_strings = ["hello", "world", "python", "hash"]
for s in test_strings:
    print(f"Hash of '{s}': {hash_function(s)}")


Hash of 'hello': 17
Hash of 'world': 34
Hash of 'python': 91
Hash of 'hash': 15
